# Compute component maps with FGB routines

With the previous analysis on 2 $\beta$ model, we saw that FGB estimate well parameters with BI than CMB-S4. Otherwise, there is a bug on FGB pipeline, we don't know why but the maps separated are not well estimated but parameters yes. This notebook try to estimate those maps correctly.

In [ ]:
import qubicplus
import pysm3
import pysm3.units as u
from pysm3 import utils
import numpy as np
from qubic import camb_interface as qc
import healpy as hp
import matplotlib.pyplot as plt
import os
import random as rd
import string
import qubic
from importlib import reload
import pickle
import s4bi
from scipy import constants
import fgbuster

center = qubic.equ2gal(0, -57)
# If there is not this command, the kernel shut down every time..
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

nside=256
def get_coverage(fsky, nside, center_radec=[0., -57.]):
    center = qubic.equ2gal(center_radec[0], center_radec[1])
    uvcenter = np.array(hp.ang2vec(center[0], center[1], lonlat=True))
    uvpix = np.array(hp.pix2vec(nside, np.arange(12*nside**2)))
    ang = np.arccos(np.dot(uvcenter, uvpix))
    indices = np.argsort(ang)
    okpix = ang < -1
    okpix[indices[0:int(fsky * 12*nside**2)]] = True
    mask = np.zeros(12*nside**2)
    mask[okpix] = 1
    return mask

covmap = get_coverage(0.03, nside)

def separate(comp, instr, maps_to_separate, tol=1e-5, print_option=False):
    solver_options = {}
    solver_options['disp'] = False
    fg_args = comp, instr, maps_to_separate
    fg_kwargs = {'method': 'TNC', 'tol': tol, 'options': solver_options}
    try:
        res = fgbuster.basic_comp_sep(*fg_args, **fg_kwargs)
    except KeyError:
        fg_kwargs['options']['disp'] = False
        res = fgbuster.basic_comp_sep(*fg_args, **fg_kwargs)
    if print_option:
        print()
        print("message:", res.message)
        print("success:", res.success)
        print("result:", res.x)
        
    return res

# Loading of dictionary from BI-CMBS4.ipynb 

### CMB-S4 config

freqs = np.array([20., 30., 40., 85., 95., 145., 155., 220., 270.])
bandwidth = np.array([5., 9., 12., 20.4, 22.8, 31.9, 34.1, 48.4, 59.4])
dnu_nu = bandwidth/freqs
beam_fwhm = np.array([11., 72.8, 72.8, 25.5, 25.5, 22.7, 22.7, 13., 13.])
mukarcmin_TT = np.array([16.5, 9.36, 11.85, 2.02, 1.78, 3.89, 4.16, 10.15, 17.4])
mukarcmin_EE = np.array([10.87, 6.2, 7.85, 1.34, 1.18, 1.8, 1.93, 4.71, 8.08])
mukarcmin_BB = np.array([10.23, 5.85, 7.4, 1.27, 1.12, 1.76, 1.89, 4.6, 7.89])
ell_min = np.array([30, 30, 30, 30, 30, 30, 30, 30, 30])
nside = np.array([256, 256, 256, 256, 256, 256, 256, 256, 256])
edges_min = freqs * (1. - dnu_nu/2)
edges_max = freqs * (1. + dnu_nu/2)
edges = [[edges_min[i], edges_max[i]] for i in range(len(freqs))]
s4_config = {
    'nbands': len(freqs),
    'frequency': freqs,
    'depth_p': 0.5*(mukarcmin_EE + mukarcmin_BB),
    'depth_i': mukarcmin_TT,
    'depth_e': mukarcmin_EE,
    'depth_b': mukarcmin_BB,
    'fwhm': beam_fwhm,
    'bandwidth': bandwidth,
    'dnu_nu': dnu_nu,
    'ell_min': ell_min,
    'nside': nside,
    'fsky': 0.03,
    'ntubes': 12,
    'nyears': 7.,
    'edges': edges,
    'effective_fraction': np.zeros(len(freqs))+1.
            }

def fct_subopt(nus):
    subnus = [150., 220]
    subval = [1.4, 1.2]
    fct_subopt = np.poly1d(np.polyfit(subnus, subval, 1))
    return fct_subopt(nus)

subnus = [150., 220]
subval = [1.4, 1.2]

def qubicify(config, qp_nsub, qp_effective_fraction):
    nbands = np.sum(qp_nsubs)
    qp_config = config.copy()
    for k in qp_config.keys():
        qp_config[k]=[]
    qp_config['nbands'] = nbands
    qp_config['fsky'] = config['fsky']
    qp_config['ntubes'] = config['ntubes']
    qp_config['nyears'] = config['nyears']
    qp_config['initial_band'] = []

    for i in range(len(config['frequency'])):
        #print(config['edges'][i][0], config['edges'][i][-1])
        newedges = np.linspace(config['edges'][i][0], config['edges'][i][-1], qp_nsubs[i]+1)
        #print(newedges)
        newfreqs = (newedges[0:-1]+newedges[1:])/2
        newbandwidth = newedges[1:] - newedges[0:-1]
        newdnu_nu = newbandwidth / newfreqs
        newfwhm = config['fwhm'][i] * config['frequency'][i]/newfreqs
        scalefactor_noise = np.sqrt(qp_nsubs[i]) * fct_subopt(config['frequency'][i]) / qp_effective_fraction[i]
        newdepth_p = config['depth_p'][i] * np.ones(qp_nsub[i]) * scalefactor_noise
        newdepth_i = config['depth_i'][i] * np.ones(qp_nsub[i]) * scalefactor_noise
        newdepth_e = config['depth_e'][i] * np.ones(qp_nsub[i]) * scalefactor_noise
        newdepth_b = config['depth_b'][i] * np.ones(qp_nsub[i]) * scalefactor_noise
        newell_min = np.ones(qp_nsub[i]) * config['ell_min'][i]
        newnside = np.ones(qp_nsub[i]) * config['nside'][i]
        neweffective_fraction = np.ones(qp_nsub[i]) * qp_effective_fraction[i]
        initial_band = np.ones(qp_nsub[i]) * config['frequency'][i]

        for k in range(qp_nsubs[i]):
            if qp_effective_fraction[i] != 0:
                qp_config['frequency'].append(newfreqs[k])
                if i >= 3:
                    qp_config['depth_p'].append(newdepth_p[k])
                    qp_config['depth_i'].append(newdepth_i[k])
                    qp_config['depth_e'].append(newdepth_e[k])
                    qp_config['depth_b'].append(newdepth_b[k])
                else:
                    qp_config['depth_p'].append(s4_config['depth_p'][i])
                    qp_config['depth_i'].append(s4_config['depth_i'][i])
                    qp_config['depth_e'].append(s4_config['depth_e'][i])
                    qp_config['depth_b'].append(s4_config['depth_b'][i])
                qp_config['fwhm'].append(newfwhm[k])
                qp_config['bandwidth'].append(newbandwidth[k])
                qp_config['dnu_nu'].append(newdnu_nu[k])
                qp_config['ell_min'].append(newell_min[k])
                qp_config['nside'].append(newnside[k])

                qp_config['effective_fraction'].append(neweffective_fraction[k])
                qp_config['initial_band'].append(initial_band[k])
        for k in range(qp_nsubs[i]+1):
            if qp_effective_fraction[i] != 0:
                qp_config['edges'].append(newedges[k])

        #qp_config['depth_p'][:3] = s4_config['depth_p'][:3]
        #qp_config['depth_i'][:3] = s4_config['depth_i'][:3]

    fields = ['frequency', 'depth_p', 'depth_i', 'depth_e', 'depth_b', 'fwhm', 'bandwidth',
              'dnu_nu', 'ell_min', 'nside', 'edges', 'effective_fraction', 'initial_band']
    for j in range(len(fields)):
        qp_config[fields[j]] = np.array(qp_config[fields[j]])

    return qp_config

thr = 0
mymask = (covmap > (np.max(covmap)*thr)).astype(int)
pixok = mymask > 0

qp_nsubs = np.array([1, 1, 1, 1, 1, 1, 1, 5, 5])
qp_effective_fraction = np.array([1, 1, 1, 1, 1, 1, 1, 1, 1])
qp_config=qubicify(s4_config, qp_nsubs, qp_effective_fraction)

#Corrected depths
qp_config['depth_p'][:7] = s4_config['depth_p'][:7]
qp_config['depth_i'][:7] = s4_config['depth_i'][:7]

plt.figure(figsize=(16, 5))
plt.subplot(1,2,1)
plt.errorbar(s4_config['frequency'], s4_config['depth_p'], xerr=s4_config['bandwidth']/2, fmt='ro', label='CMB-S4')
plt.errorbar(qp_config['frequency'], qp_config['depth_p'], xerr=qp_config['bandwidth']/2, fmt='bo', label='BI')
plt.xlabel('Frequency [GHz]')
plt.ylabel(r'Depth_p [$\mu$K.arcmin]')
plt.title('CMB-S4 Configuration')
plt.legend()
plt.subplot(1,2,2)
plt.errorbar(s4_config['frequency'], s4_config['fwhm'], xerr=s4_config['bandwidth']/2, fmt='ro', label='CMB-S4')
plt.errorbar(qp_config['frequency'], qp_config['fwhm'], xerr=qp_config['bandwidth']/2, fmt='bo', label='BI')
plt.xlabel('Frequency [GHz]')
plt.ylabel('FWHM [arcmin]')
plt.title('CMB-S4 Configuration')
plt.legend()
plt.show()

In [ ]:
map_dust_bi, map_dust_bi_noiseless, map_dust_s4, map_dust_s4_noiseless = get_maps({'dust':'d02b'}, covmap)

In [ ]:
map_cmb_bi, map_cmb_bi_noiseless, map_cmb_s4, map_cmb_s4_noiseless = get_maps({'cmb':42}, covmap)

In [ ]:
def paramCompSep(config, name, noisy=True, skyconfig={'cmb':42, 'dust':'d02b', 'synchrotron':'s0'}, verbose=False):
    
    nside=256
    covmap = get_coverage(0.03, nside)
    thr = 0
    mymask = (covmap > (np.max(covmap)*thr)).astype(int)
    pixok = mymask > 0
    beta=[1.34, 1.74, 265]
    nu0=145
    
    print('######################')
    print('nu0 = {:.0f} GHz'.format(nu0))
    print('beta0 = {:.3f}'.format(beta[0]))
    print('beta1 = {:.3f}'.format(beta[1]))
    print('nubreak = {:.3f} GHz'.format(beta[2]))
    print('sky fraction = {:.2f} %'.format(0.03*100))
    print('###################### \n \n')
    
    ### Generate maps
    
    print("Maps generation")
    
    if noisy:
        maps_to_separate, _, _ = qubicplus.BImaps(skyconfig, config, r=0).getskymaps(same_resol=0, 
                                      verbose=verbose, 
                                      coverage=covmap, 
                                      noise=noisy,
                                      beta=beta)
    else:
        maps_to_separate = qubicplus.BImaps(skyconfig, config, r=0).getskymaps(same_resol=0, 
                                      verbose=verbose, 
                                      coverage=covmap, 
                                      noise=noisy,
                                      beta=beta)
    
    print("Initialize instrument")
    instr=fgbuster.observation_helpers.get_instrument(name)
    instr.frequency = config['frequency']
    instr.fwhm = config['fwhm']
    instr.depth_i = config['depth_i']
    instr.depth_p = config['depth_p']
    
    # Define components
    print("Define components")
    comp=[fgbuster.component_model.Dust_2b(nu0=353), 
          fgbuster.component_model.CMB(),
          fgbuster.component_model.Synchrotron(nu0=70)]
    
    
    options={'disp':False} 
    tol=1e-5
    method='TNC'
    
    fg_args = comp, instr, maps_to_separate[:, :, pixok]
    fg_kwargs = {'method':method, 'tol':tol, 'options':options}
    print('Separation')
    res = fgbuster.basic_comp_sep(*fg_args, **fg_kwargs)
    
    #print(res.x)

    print('\nFit of spectral indices -> ', res.x)
    print('Estimated error bar on spectral indices -> ', np.diag(res.Sigma))
    
    print('Estimation of Mixing Matrix')
    # Estimation of mixing matrix
    A = fgbuster.mixingmatrix.MixingMatrix(*comp)
    A_ev = A.evaluator(instr.frequency)
    # Mixing matrix evaluation at max L
    A_maxL = A_ev(np.round(res.x, 3))
    
    
    # pixel seen
    ind=np.where(pixok != 0)[0]
    mysolution=np.ones(((3, 3, 12*nside**2)))*hp.UNSEEN
    if noisy:
        
        invN = np.diag(hp.nside2resol(256, arcmin=True) / (instr.depth_p))**2
        inv_AtNA = np.linalg.inv(A_maxL.T.dot(invN).dot(A_maxL))
        
        # Loop over pixels
        for i in ind:
            # Loop over stokes parameters
            inv_AtNA_dot_At_dot_invN=inv_AtNA.dot(A_maxL.T).dot(invN)
            for s in range(3):
                mysolution[:, s, i] = inv_AtNA_dot_At_dot_invN.dot(maps_to_separate[:, s, i])
        #print('Shape of inv_AtNA_dot_At_dot_invN -> ', inv_AtNA_dot_At_dot_invN.shape)
    else:
        print('\n          >>> building s = Wd in pixel space \n')
        mysol = fgbuster.algebra.Wd(A_maxL, maps_to_separate[:, :, pixok].T).T
        mysolution[:, :, pixok]=mysol.copy()
    
    # Normalization
    ind_nu=np.where(config['frequency']==nu0)[0][0]
    
    for c in range(len(comp)):
        mysolution[c, :, :]*=A_maxL[ind_nu, c]
        
    return mysolution

In [ ]:
sols4=paramCompSep(s4_config, 'CMBS4', noisy=False, verbose=False)
solbi=paramCompSep(qp_config, 'Qubic+', noisy=False, verbose=False)

In [ ]:
sols4[:, :, ~pixok]=hp.UNSEEN
solbi[:, :, ~pixok]=hp.UNSEEN

ind_nu=5
c=1
istk=1

plt.figure(figsize=(16, 10))
hp.gnomview(map_cmb_s4_noiseless[ind_nu, istk, :], rot=center, reso=15, cmap='jet', sub=(2, 3, 1), title='True')
hp.gnomview(sols4[c, istk, :], rot=center, reso=15, cmap='jet', sub=(2, 3, 2), title='S4')
hp.gnomview(solbi[c, istk, :], rot=center, reso=15, cmap='jet', sub=(2, 3, 3), title='BI')

res_s4=sols4[c, istk, :]-map_cmb_s4_noiseless[ind_nu, istk, :]
res_bi=solbi[c, istk, :]-map_cmb_s4_noiseless[ind_nu, istk, :]

res_s4[~pixok]=hp.UNSEEN
res_bi[~pixok]=hp.UNSEEN

hp.gnomview(res_s4[:], rot=center, reso=15, cmap='jet', sub=(2, 3, 5), title='S4 - True -> RMS : {:.7f}'.format(
    np.std(res_s4[pixok])))
hp.gnomview(res_bi[:], rot=center, reso=15, cmap='jet', sub=(2, 3, 6), title='BI - True -> RMS : {:.7f}'.format(
    np.std(res_bi[pixok])))
plt.show()